# ECON 622
# Problem Set 4
*Nicolas Wesseler*

This is a failing attempt at solving the optimal control problem. 

# Model Description

 This is based on section II of the paper.

  ## Households

$$
 \begin{aligned}
  \max_{c_t, \ell_t, a_t, b_t, d_t} E_0 & \int_0^\infty
 e^{−(ρ+ζ)t} u( c_t, \ell_t) dt \\
 \text{s.t.} \\
 \frac{db}{dt} & = (1 − τ_t)w_t z_t ℓ_t + r^b_t(b_t) b_t + T_t − d_t − \chi(d_t, a_t) − c_t, \\
 \frac{da}{dt} & = r_t^a a_t + d_t \\
 b_t & \geq −b_ ,\;\; a_t \geq 0
 \end{aligned}
$$
  where $a_0$, $b_0$ and $d_0$ are given, and
 $$
  r^b_t(b_t) = r_t^b + \kappa
 $$
  and
$$
 \chi(d_t, a_t) =  \chi_0 |d_t| - \chi_1 \left\vert \frac{d_t}{a_t}\right \vert^{\chi_2} a
$$

### Activate environment

In [23]:
cd("/Users/nicolaswesseler/Documents/github/ECON622/problem_sets");
using Pkg
Pkg.activate

activate (generic function with 3 methods)

### Install missing packages

In [24]:
#Pkg.add("LinearAlgebra")
#Pkg.add("NLsolve")
#Pkg.add("EconPDEs")

### Load packages

In [4]:
using EconPDEs
using NLsolve
using LinearAlgebra

In [ ]:
# Define a discretized state space
# An OrderedDict in which each key corresponds to a state variable
# Grids can be non-homogeneous

stategrid = OrderedDict(:ab => range(0.0, stop=10.0, length=100), :bb => range(-10.0, stop=10.0, length=200), :xb => range(0, stop=2, length=100))

In [41]:
# Define an initial guess for the value functions
# An OrderedDict in which each key corresponds to a value function to solve for, 
# specified as an array with as many dimensions as there are state variables

solend = OrderedDict(:V => ones(100))

OrderedDict{Symbol, Vector{Float64}} with 1 entry:
  :V => [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.…

In [1]:
# Define a function that encodes the PDE. 

function f(state::NamedTuple, sol::NamedTuple)
    χ_0 = 0.04; χ_1 = 0.8; χ_2 = 0.9; ϕ = 3; w = 2; T = 0.0; ra = 0.1; rb = 0.04; β = 0.1; ρ = 0.04; λ = 0; ψ = 2; v = 1

    # create state variables
    g = ["a", "b", "x"] 
    for i in g
        i = state.i
    end

    # use the solver
    V, Va, Va_d, Vb, Vb_d, Vx, Vx_d = sol.V, sol.Va_up, sol.Va_down, sol.Vb_up, sol.Vb_down, sol.Vx_up, sol.Vx_down 
    l = ((1/ϕ)* Vb* (1-τ) *w * (e^x))^(1/v)
    c = (Vb)^(-1/ψ)
    d = ((Va /Vb - 1) - χ_0)/(χ_1*χ_2* a^(1-χ_2))^(1/(χ_2-1)) > 0 ? ((Va /Vb - 1) - χ_0)/(χ_1*χ_2* a^(1-χ_2))^(1/(χ_2-1)) : ((Va /Vb - 1) * (-1) - χ_0)/(χ_1*χ_2* a^(1-χ_2))^(1/(χ_2-1))
    χ = χ_0 * abs(d) + χ_1*((abs(d/a))^χ_2)*a

    # solution
    Vt = - ( (c^(1-x))/(1-x) - ϕ * ((l^(1+v))/(1+v)) + Vb * ((1-τ)*w* exp(y) *l + r_b * b +T - d - χ -c) + Va * (r_a*a + d) + Vx*(-β*x) - (ρ + ζ)*V )
    (Vt = Vt,), (; V, Va, Vb, Vx)
end

f (generic function with 1 method)

In [ ]:
# Solve directly for the stationary solution

x, residual_norm =  pdesolve(f, stategrid, solend)